In [12]:
import json
import math
import networkx as nx
import pandas as pd
import altair as alt
import numpy as np
from scipy import stats
from scipy.optimize import curve_fit
import distfit
import os
import timeit


from generator_v2 import Generator

alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

In [121]:
# define the theme by returning the dictionary of configurations

font = "monospace"
# Axes
axisColor = "#000000"
gridColor = "#DEDDDD"
# Colors
main_palette = ["#1696d2", 
                "#d2d2d2",
                "#000000", 
                "#fdbf11", 
                "#ec008b", 
                "#55b748", 
                "#5c5859", 
                "#db2b27", 
               ]
sequential_palette = ["black", 
                      "#a2d4ec", 
                      "#73bfe2", 
                      "#46abdb", 
                      "#1696d2", 
                      "#12719e", 
                     ]


# markColor = "#8856a7"
# markColor = "#6a51a3"
markColor = "#9ebcda"

# Define custom color schemes for ordinal and categorical data
categorical_color_scheme = ['red', 'green', 'blue', 'purple', 'orange']
ordinal_color_scheme = ["#fcfbfd",
"#efedf5",
"#dadaeb",
"#bcbddc",
"#9e9ac8",
"#807dba",
"#6a51a3",
"#4a1486"]


def simple_theme():
    return {
        'config': {
            'view': {
                'height': 300,
                'width': 300,
            },
            "title": {
                    "fontSize": 18,
                    "font": font,
                    "anchor": "start", # equivalent of left-aligned.
                    "fontColor": "#000000"
                },
            "facet": {
                "labelFontSize": 16,
                "titleFontSize": 20  # Adjust the fontsize as needed
            },
            "axisX": {
                    "labelFont": font,
                    "labelFontSize": 16,
                    "titleFontSize": 20,
                    "grid": False,
                },
                "axisY": {
                    "domain": False,
                    "grid": False,
                    "labelFont": font,
                    "labelFontSize": 16,
                    "titleFontSize": 20,
                },
            'mark': {
                'color': markColor,
                'fill': markColor
            },
            "line": {
                "fill": None,
               "stroke": "black",
           },
            "scale": {"color": {"scheme": "blue"}},
            "legend": {

                "titleFontSize": 20,
                "labelFontSize": 16
            }            
        }
    }

# alt.themes.register("theme", theme)
alt.themes.register("theme", simple_theme)
alt.themes.enable("theme")
#alt.themes.enable("ggplot2")

ThemeRegistry.enable('theme')

## N Fixed

In [140]:
N = 5000
N_coms = 4
p = 100 / N
q = 0.1 * p

Es = [10, 100, 500, 1000, 2000, 5000]

community_array = []
for i in range(N_coms):
    community_array += [i for x in range(N // N_coms)]

In [141]:
p

0.02

In [60]:
times = []
for E in Es:
    print(E)

    start = timeit.default_timer()
    gen = Generator(N, E, N_coms, p, q, community_array, "weighted")
    gen.run()
    stop = timeit.default_timer()

    time = stop - start
    times.append(time)
    print(time)



10
1.3446321619994706
100
1.8332682329928502
500
4.023058003993356
1000
6.842099884990603
2000
12.573685010007466
5000
30.315783561993157


In [63]:
df_N_fixed = pd.DataFrame({"E": Es, "time": times}) 

In [64]:
df_N_fixed.head()

,E,time
0,10,1.344632
1,100,1.833268
2,500,4.023058
3,1000,6.842100
4,2000,12.573685


In [123]:
alt.Chart(df_N_fixed).mark_line().encode(
    x=alt.X('E:Q'),
    y=alt.Y('time:Q'),
)

alt.Chart(...)

# Change N and E

In [72]:
N_coms = 4
p = 100 / N
q = 0.1 * p

Es = [200, 500, 1000, 2000, 3000, 4000, 5000]
Ns = [200, 500, 1000, 2000, 3000, 4000, 5000]

In [73]:
times = []
for N, E in zip(Ns, Es):
    print(N, E)

    community_array = []
    for i in range(N_coms):
        community_array += [i for x in range(N // N_coms)]

    start = timeit.default_timer()
    gen = Generator(N, E, N_coms, p, q, community_array, "weighted")
    gen.run()
    stop = timeit.default_timer()

    time = stop - start
    times.append(time)
    print(time)

200 200
0.02366244900622405
500 500
0.1685958199959714
1000 1000
0.5476375879952684
2000 2000
2.5114922559878323
3000 3000
6.43221969800652
4000 4000
12.797013485993375
5000 5000
21.035206211003242


In [74]:
df_NE = pd.DataFrame({"E": Es, "N": Ns, "time": times}) 

In [126]:
chart = alt.Chart(df_NE).mark_line(color=markColor, opacity=1, size=3).encode(
    x=alt.X('N:Q'),
    y=alt.Y('time:Q', title="Time (s)"),
)
chart

alt.Chart(...)

In [89]:
chart + chart.transform_regression('N', 'time', method="quad").mark_line()

alt.LayerChart(...)

In [127]:
# degree 2 polynomial fit or quadratic fit 
model = np.poly1d(np.polyfit(df_NE['N'], 
                             df_NE['time'], 2)) 

In [128]:
model.coefficients

array([ 1.02301370e-06, -9.82813467e-04,  3.40510243e-01])

In [129]:
def apply_fit(x):
    return model.coefficients[0] * x**2 + model.coefficients[1] * x + model.coefficients[2]

df_NE["fit"] = df_NE["N"].apply(apply_fit)

In [138]:
line = alt.Chart(df_NE).mark_line(opacity=1, size=3, color=markColor).encode(
    x=alt.X('fit:Q'),
    y=alt.Y('time:Q'),
)

point = alt.Chart(df_NE).mark_point(
    size=80, opacity=0.8, stroke="black", shape="triangle").encode(
    x=alt.X("fit", title="1.01E-6N^2 - 9.82E-4N + 3.40E-1"),
    y=alt.Y(
        "time",
        title="Time (s)",
    )
)

In [139]:
line + point

alt.LayerChart(...)

In [84]:
model.r

array([480.35205739+319.54973377j, 480.35205739-319.54973377j])

In [ ]:
model = np.poly1d(np.polyfit(hours, happ, 2))

In [ ]:
alt.Chart(df_NE).mark_line().encode(
    x=alt.X('N:Q'),
    y=alt.Y('time:Q'),
)